In [47]:
from scipy.io import loadmat
data = loadmat('trainingData.mat')
data = data['dataRecord']


In [48]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
torch.backends.quantized.engine = 'qnnpack'
# 假设你的数据是numpy数组形式，需要转换成torch Tensor
import numpy as np

# 示例数据，你应该用你的实际数据替换这里的np.random.rand()
inputs = torch.tensor(data[:, :6], dtype=torch.float32)
outputs = torch.tensor(data[:, 6:], dtype=torch.float32)

# 创建数据集和数据加载器
dataset = TensorDataset(inputs, outputs)
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

# 定义神经网络模型
class SimpleFNN(nn.Module):
    def __init__(self):
        super(SimpleFNN, self).__init__()
        self.fc1 = nn.Linear(6, 6)  # 输入层到隐藏层，50个神经元
        self.fc2 = nn.Linear(6, 3)  # 输出层

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

model = SimpleFNN()
model.train()

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练网络
epochs = 200
for epoch in range(epochs):
    for inputs, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')


Epoch 1, Loss: 0.045155249536037445
Epoch 2, Loss: 0.020653136074543
Epoch 3, Loss: 0.008802822791039944
Epoch 4, Loss: 0.0030329697765409946
Epoch 5, Loss: 0.0010595686035230756
Epoch 6, Loss: 0.0003137744788546115
Epoch 7, Loss: 7.538412319263443e-05
Epoch 8, Loss: 2.7716212571249343e-05
Epoch 9, Loss: 1.4252132132241968e-05
Epoch 10, Loss: 1.0664414730854332e-05
Epoch 11, Loss: 1.4374438251252286e-05
Epoch 12, Loss: 1.5849725969019346e-05
Epoch 13, Loss: 9.750559911481105e-06
Epoch 14, Loss: 5.980773948977003e-06
Epoch 15, Loss: 1.806268483051099e-05
Epoch 16, Loss: 2.0705094357253984e-05
Epoch 17, Loss: 7.319287306017941e-06
Epoch 18, Loss: 1.5876681572990492e-05
Epoch 19, Loss: 9.442064765607938e-06
Epoch 20, Loss: 1.984913797059562e-05
Epoch 21, Loss: 1.2657120350922924e-05
Epoch 22, Loss: 1.1222474313399289e-05
Epoch 23, Loss: 1.2641274224733934e-05
Epoch 24, Loss: 9.524382221570704e-06
Epoch 25, Loss: 8.372232514375355e-06
Epoch 26, Loss: 1.0114022188645322e-05
Epoch 27, Loss: 

In [49]:
# infer and save
test_input = torch.tensor([[0.11412254, -0.09698317, -0.13648682, -0.08168529,  0.09584415,
        -0.08081015]], dtype=torch.float32)  # 示例输入数据
with torch.no_grad():
    prediction = model(test_input)
    print(f'Prediction: {prediction.numpy()}')
    print("-0.01443261, -0.01519005, -0.02687549")
# save the model
scripted_model = torch.jit.script(model)
torch.jit.save(scripted_model, '../model/model.pth')

Prediction: [[-0.01569352 -0.01485252 -0.02799098]]
-0.01443261, -0.01519005, -0.02687549


In [51]:
data = loadmat('testingData.mat')
test_data = data['dataRecord']

# 测试数据集，应该与训练数据集具有相同的结构
test_inputs = torch.tensor(test_data[:, :6], dtype=torch.float32)
test_outputs = torch.tensor(test_data[:, 6:], dtype=torch.float32)

# 创建测试 DataLoader
test_dataset = TensorDataset(test_inputs, test_outputs)
test_loader = DataLoader(test_dataset, batch_size=10)

# 将模型设置为评估模式
model.eval()

# 初始化损失计算
test_loss = 0.0
with torch.no_grad():  # 确保不进行梯度计算
    for inputs, labels in test_loader:
        # 前向传播
        outputs = model(inputs)
        # 计算损失
        loss = criterion(outputs, labels)
        test_loss += loss.item()

# 平均损失
test_loss /= len(test_loader)
print(f'Average loss on the test dataset: {test_loss}')


Average loss on the test dataset: 2.3508601771027316e-06


In [56]:
qm = torch.quantization.quantize_dynamic(model, {nn.Linear}, dtype=torch.qint8)
qm.eval()

# 初始化损失计算
test_loss = 0.0
with torch.no_grad():  # 确保不进行梯度计算
    for inputs, labels in test_loader:
        # 前向传播
        outputs = qm(inputs)
        # 计算损失
        loss = criterion(outputs, labels)
        test_loss += loss.item()

# 平均损失
test_loss /= len(test_loader)
print(f'Average loss on the test dataset: {test_loss}')
scripted_qmodel = torch.jit.script(qm)
torch.jit.save(scripted_qmodel, '../model/qmodel.pth')

Average loss on the test dataset: 2.1326906335161765e-06


[W qlinear_dynamic.cpp:247] Warning: Currently, qnnpack incorrectly ignores reduce_range when it is set to true; this may change in a future release. (function operator())
